In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.models import Sequential
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs as go

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 4007)


In [2]:
# Set a random seed to reproduce the results
np.random.seed(1234)

# Load the volume data
volume_data = pd.read_csv('../data/volume_data.csv', header=None)

In [3]:
all_hfs = pd.read_csv('../data/hf_list.csv')

used_hfs = [16911, 16912, 16913, 278, 10528, 16515, 14479, 16551, 6297, 16537]

def find_index_hf(hf_no):
    return all_hfs[all_hfs['HF'] == hf_no].index.tolist()[0]

hfs = [find_index_hf(hf_no) for hf_no in used_hfs]

In [4]:
maxs = []
mins = []
means = []
stds = []

In [5]:
def train_test_traffic_data(sequence_length=50, horizon=15):
    global maxs
    global mins
    global means
    global stds
    maxs = []
    mins = []
    means = []
    stds = []
     # for 30 minutes aggregate divide by 2
    sample_size = int(volume_data.shape[0] * (15/horizon))
    result = []
    # scale data
    # Create data
    for j in hfs:
        road_vol = volume_data[j]
        road_vol = road_vol.replace(0, np.nan) 
        road_vol = road_vol.interpolate().values 
        
        if horizon == 30:
            road_vol = road_vol[0::2] + road_vol[1::2] 
        elif horizon == 45:
            road_vol = road_vol[0::3] + road_vol[1::3] + road_vol[2::3]
        # for 30 minutes aggregate
        # road_vol = road_vol[0::2] + road_vol[1::2]
        
        mean_t = road_vol.mean()
        max_t = road_vol.max()
        min_t = road_vol.min()
        std_t = np.std(road_vol)
        
        mins.append(min_t)
        maxs.append(max_t)
        means.append(mean_t)
        stds.append(std_t)
        
        road_vol = (road_vol - mean_t) / std_t
        temp = []
        for i in range(0, sample_size - sequence_length):
            temp.append(road_vol[i: i + sequence_length])
        result.append(temp)

    result = np.dstack(result)
    #row = round(0.9 * result.shape[0])
    row = result.shape[0] - 5280
    train = result[:row, :, :]
    np.random.shuffle(train)
    X_train = train[:, :-1, :]
    y_train = train[:, -1, :]
    X_test = result[row:, :-1, :]
    y_test = result[row:, -1, :]

    return [X_train, y_train, X_test, y_test]

In [6]:
def lstm_model():
    mdl = Sequential()
    io_dim = len(hfs)
    # a network with len(hfs)-dimensional input,
    # 3 hidden layers of sizes 100, 200, 200
    # and eventually a len(hfs)-dimensional output layer
    layers = [io_dim, 200, 200, io_dim]

    # We also add 20% Dropout in this layer.
    mdl.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.2))
    
    # 3rd hidden layer
    mdl.add(LSTM(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.2))

    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [7]:
def simple_rnn_model():
    mdl = Sequential()
    io_dim = len(hfs)
    # a network with 1-dimensional input,
    # two hidden layers of sizes 100 and 100
    # and eventually a 1-dimensional output layer
    layers = [io_dim, 200, 200, io_dim]

    # We also add 10% Dropout in this layer.
    mdl.add(SimpleRNN(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.1))

    # 3rd hidden layer
    mdl.add(SimpleRNN(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.2))
    
    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [8]:
def gru_model():
    mdl = Sequential()
    io_dim = len(hfs)
    # a network with 1-dimensional input,
    # two hidden layers of sizes 50 and 100
    # and eventually a 1-dimensional output layer
    layers = [io_dim, 200, 200, io_dim]

    # We also add 20% Dropout in this layer.
    mdl.add(GRU(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    mdl.add(Dropout(0.2))

    # 2nd hidden layer
    mdl.add(GRU(
        layers[2],
        return_sequences=False))
    mdl.add(Dropout(0.2))
    
    # last layer we use is a Dense layer ( = feedforward).
    # Since we are doing a regression, its activation is linear
    mdl.add(Dense(
        output_dim=layers[3]))
    mdl.add(Activation("linear"))

    start = time.time()
    mdl.compile(loss="mse", optimizer="adam")
    print("Compilation Time : ", time.time() - start)
    return mdl

In [9]:
def run_network(data, mdl_type):
    global_start_time = time.time()
    epochs = 20

    X_train, y_train, X_test, y_test = data

    print('\nData Loaded. Compiling...\n')

    if mdl_type == 'rnn':
        mdl = simple_rnn_model()
    elif mdl_type == 'gru':
        mdl = gru_model()
    elif mdl_type == 'lstm':
        mdl = lstm_model()
    else:
        return

    try:
        mdl.fit(X_train, y_train, batch_size=512,
                nb_epoch=epochs, validation_split=0.05)
        predicted_trffic = mdl.predict(X_test)
    except KeyboardInterrupt:
        print('Training duration (s) : ', time.time() - global_start_time)
        return mdl, y_test, 0

    print('Training duration (s) : ', time.time() - global_start_time)

    return mdl, y_test, predicted_trffic

In [10]:
def plot_predictions(y_test, predicted, horizon):
    y_test_tp = np.transpose(y_test)
    pred_tp = np.transpose(predicted)
   
    i = 2 #hf no 16913
    actual = (y_test_tp[i] * stds[i]) + means[i]
    predictions = (pred_tp[i] * stds[i]) + means[i]
   
    actual = actual[:2880]
    predictions = predictions[:2880]
    
    t = pd.date_range('6/1/2012', freq=(str(horizon)+'Min'), periods=(30*96*15)/horizon)
    trace0 = go.Scatter(
        x = t,
        y = actual,
        name = 'Actual')
    trace1 = go.Scatter(
        x = t,
        y = predictions,
        name = 'Predicted')
    data = [trace0, trace1]

    layout = dict(xaxis = dict(title = 'Time'),
                  yaxis = dict(title = 'Volume'),
                  width = 600, height = 450)

    fig = dict(data=data, layout=layout)
    plotly.offline.plot(fig)

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def print_scores(y_test, predicted):
    y_test_tp = np.transpose(y_test)
    pred_tp = np.transpose(predicted)
    for i in range(0, len(used_hfs)):
        mae = []
        mse = []
        mape = []
        print("Score for location -- ", used_hfs[i])
        actual = y_test_tp[i]
        predictions = pred_tp[i]

        actual = actual[:2880]
        predictions = predictions[:2880]

        actual = (actual * stds[i]) + means[i]
        predictions = (predictions * stds[i]) + means[i]

        mae.append(mean_absolute_error(actual, predictions))
        mse.append(mean_squared_error(actual, predictions))
        mape.append(mean_absolute_percentage_error(actual, predictions))
        print("MAE=", np.array(mae).mean())
        print("RMSE=", math.sqrt(np.array(mse).mean()))
        print("MAPE=", np.array(mape).mean())

In [12]:
data = train_test_traffic_data(50,15)

In [ ]:
#Using ADAM
model, y_test, predicted = run_network(data, 'lstm')
plot_predictions(y_test, predicted, 15)


Data Loaded. Compiling...

Compilation Time :  0.14113426208496094
Train on 180346 samples, validate on 9492 samples
Epoch 1/20
115200/180346 [==================>...........] - ETA: 45s - loss: 0.1143

In [ ]:
print_scores(y_test, predicted)

In [ ]:
print_scores(y_test, predicted)

In [ ]:
from keras.utils.visualize_util import plot
plot(model, to_file='../latex-thesis/Figures/lstm_multi_variate.png')

In [ ]:
model, y_test, predicted = run_network(data, 'rnn')
plot_predictions(y_test, predicted, 15)
print_scores(y_test, predicted)

In [ ]:
model, y_test, predicted = run_network(data, 'gru')
plot_predictions(y_test, predicted, 15)
print_scores(y_test, predicted)

In [ ]:
data = train_test_traffic_data(50,30)

In [ ]:
model, y_test, predicted = run_network(data, 'lstm')
plot_predictions(y_test, predicted, 30)
print_scores(y_test, predicted)

In [ ]:
model, y_test, predicted = run_network(data, 'rnn')
plot_predictions(y_test, predicted, 30)
print_scores(y_test, predicted)

In [ ]:
model, y_test, predicted = run_network(data, 'gru')
plot_predictions(y_test, predicted, 30)
print_scores(y_test, predicted)

In [ ]:
data = train_test_traffic_data(50,45)

In [ ]:
model, y_test, predicted = run_network(data,'lstm')
plot_predictions(y_test, predicted, 45)
print_scores(y_test, predicted)

In [ ]:
model, y_test, predicted = run_network(data, 'rnn')
plot_predictions(y_test, predicted, 30)
print_scores(y_test, predicted)

In [ ]:
model, y_test, predicted = run_network(data, 'gru')
plot_predictions(y_test, predicted, 30)
print_scores(y_test, predicted)